#Preparacion del entorno

Instalacion de librerias en el entorno.

In [1]:
%%capture
!pip install PyPDF2 chromadb huggingface_hub llama-cpp-python wikipedia wikidataintegrator transformers sentence-transformers
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

Carga de librerias.

In [2]:
# Interacción con el sistema y usuarios
import os
import shutil
import getpass

# Tratamiento de archivos
from urllib.request import urlretrieve
from zipfile import ZipFile
from PyPDF2 import PdfReader

# Interacción con bases de datos
import csv
import chromadb
import pandas as pd
import re

# Descarga de modelos y recursos
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
from sentence_transformers import SentenceTransformer

# Acceso a Wikidata y Wikipedia
import requests
from wikidataintegrator import wdi_login
import wikipedia

Descarga del modelo de LLM Llama 2.

In [3]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin"

model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

modelo_llm = Llama(
    model_path=model_path,
    n_threads=2,
    n_batch=512,
    n_gpu_layers=128,
    n_ctx=2048)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


Descarga de modelo de embedding

In [4]:
modelo_emb = SentenceTransformer('intfloat/multilingual-e5-large')

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

Funciones para realizar embedings

In [5]:
def vectorizar_texto(text):
  embedding = modelo_emb.encode([text])
  return embedding[0].tolist()

def vectorizar_lista_texto(lista_texto):
    return [vectorizar_texto(text) for text in lista_texto]

Creacion de base de datos vectoriales

In [6]:
chroma_client = chromadb.Client()
collection_ley = chroma_client.get_or_create_collection(name='Documentacion_de_leyes_Argentinas')
clasificador = chroma_client.get_or_create_collection(name='Clasificador')
diccionariov = chroma_client.get_or_create_collection(name='Diccionario')

Descarga de archivos de github

In [7]:
github_url = "https://github.com/SalvaMrS/TP2_NLP/archive/main.zip"
zip_file_path, _ = urlretrieve(github_url, "TP2_NLP.zip")

with ZipFile(zip_file_path, "r") as zip_ref:
    zip_ref.extractall("/content")

os.remove(zip_file_path)

source_dir = "/content/TP2_NLP-main/Documentos"
target_dir = "/content/Documentos"

os.makedirs(target_dir, exist_ok=True)

for filename in os.listdir(source_dir):
    source_path = os.path.join(source_dir, filename)
    target_path = os.path.join(target_dir, filename)
    shutil.move(source_path, target_path)

#Base de datos CSV

Funciones para la creacion del archivo csv

In [8]:
def leer_pdf(archivo_pdf):
  with open(archivo_pdf, 'rb') as file:
    lector_pdf = PdfReader(file)
    texto = "".join(lector_pdf.pages[i].extract_text() for i in range(0, len(lector_pdf.pages)))
    return texto

def limpiar_diccionario(texto):
  lineas_a_eliminar = [
    "Consultor Jurídico Digital de Honduras",
    "Diccionario Jurídico Edición 2005",
    "Pedidos al 9908397"
  ]

  for linea in lineas_a_eliminar:
    texto = texto.replace(linea, "")

  texto_limpiado = texto.replace(";", ",")
  return texto_limpiado.strip()

def dividir_diccionario(texto_limpio):
  partes = texto_limpio.split("*")
  partes_divididas = [parte.split('\n', 1) for parte in partes]
  resultado = [(frase.strip(), significado.strip().replace('\n', ' ')) for frase, significado in partes_divididas[1:]]
  return resultado

def guardar_en_csv(resultados, ruta_archivo):
  with open(ruta_archivo, 'w', newline='', encoding='utf-8') as archivo_csv:
    escritor_csv = csv.writer(archivo_csv, delimiter=';')
    escritor_csv.writerow(['frase', 'significado'])
    escritor_csv.writerows(resultados)

def procesar_diccionario(archivo_pdf, ruta_csv):
  texto_extraido = leer_pdf(archivo_pdf)
  texto_limpio = limpiar_diccionario(texto_extraido)
  resultado_limpio = dividir_diccionario(texto_limpio)
  guardar_en_csv(resultado_limpio, ruta_csv)

Ejecucion

In [9]:
archivo_pdf = "/content/Documentos/Diccionario/Diccionario_juridico.pdf"
ruta_csv = "/content/Documentos/Diccionario/diccionario.csv"

procesar_diccionario(archivo_pdf, ruta_csv)

#Base de datos vectorial

Funcion para la obtencion de articulos de la constitucion, codigo civil y codigo penal argentino.

In [10]:
def segmentar_articulos(texto, doc):
  expresion = r'A[Rr][Tt].{0,8}\d{1,4}.{0,3}[-—.]'
  expresion_regular = re.compile(expresion, re.MULTILINE)

  resultado = re.split(expresion_regular, texto)
  machs = expresion_regular.findall(texto)

  resultado.pop(0)

  expresion_resultado = re.compile(r'^.*?(?=\.\n)')

  textos = []
  metadata = []

  for i, tex in enumerate(resultado):
      texto_formateado = tex.replace('\n', ' ')

      textos.append(doc + ': ' + machs[i] + texto_formateado)
      metadata.append({'articulo': machs[i], 'doc': doc})

  return textos, metadata

Obtencion de valores a guardar en la base de datos.

In [11]:
dir_data = "/content/Documentos/Leyes"

list_data = [os.path.join(dir_data, filename)
  for filename in os.listdir(dir_data)
  if not filename.startswith('.')
  and filename.lower().endswith('.pdf')
  ]

lista_textos = []
lista_metadatas = []

for archivo_path in list_data:

  texto = leer_pdf(archivo_path)
  nombre_doc = archivo_path.split("/")[-1].split("-")[0]
  textos, metadatas = segmentar_articulos(texto, nombre_doc)

  lista_textos.extend(textos)
  lista_metadatas.extend(metadatas)

lista_indices = [str(numero) for numero in range(len(lista_textos))]

Carga de articulos en la base de datos.

In [12]:
collection_ley.add(
    embeddings=vectorizar_lista_texto(lista_textos),
    documents=lista_textos,
    metadatas=lista_metadatas,
    ids=lista_indices
)

#Selector de Database


Generamos ejemplos de consultas y su correspondiente fuente de informacion.

In [13]:
lista_consultas= ["Que es una conducta ilicita?",
               'me nombraron la palabra coaccion, que es?',
               'un amigo me habló del arraigo, a que se refiere?',
               'a que se refiere cuando un abogado dice A fortiori',
               'Que es un gobierno federal?',
               'define "Grandes potencias"',
               'dime que es un Pago con subrogación',
               'que es la legitima defensa?',
               'que seria la jurisdicción voluntaria',
               'dime las caracteristicas de la dación en pago',
               '¿Cuáles son los principios fundamentales de la Ley de Servicios de Comunicación Audiovisual en Argentina?',
               'Explique los cambios introducidos por la Reforma Constitucional Argentina de 1866',
               'de que trata la Ley Diana Sacayán',
               'por que se establecio la Ley de Basura Cero?',
               'Cuando se empezó a aplicar la Ley Sáenz Peña? Por que?',
               'Explicame que dice la Ley 24.557',
               'En que consistia el Decreto 83/2015',
               'Explique los objetivos y disposiciones de la Ley Yolanda en Argentina.',
               '¿Cuáles son los objetivos de la Ley de Educación Ambiental Integral en Argentina?',
               'Describa los aspectos fundamentales de la Ley de Sociedades Comerciales en el país.',
               'que ocurre si veo un delito y no lo denuncio?',
               'Cuáles son los derechos y deberes de los padres con sus hijos?',
               'si quiero algo de una persona puedo tomarlo segun el codigo civil?',
               'Fuí victima de un delito, Qué debo hacer?',
               'Que derecho tengo a la hora de construir una casa?',
               'que puedo hacer si debo dinero a otra persona?',
               'que dice el codigo penal sobre la alimentacion de los combictos?',
               'explicame que derechos tengo frente a intrusos en mi casa',
               'dime que sucede si cometo un delito sin saberlo, y si finjo no saber nada?',
               'si me amenazan para que robe y lo hago, soy culpable?']

metadatas_consultas = [{"db": 1},{"db": 1},{"db": 1},{"db": 1},{"db": 1},
                   {"db": 1},{"db": 1},{"db": 1},{"db": 1},{"db": 1},
                   {"db": 2},{"db": 2},{"db": 2},{"db": 2},{"db": 2},
                   {"db": 2},{"db": 2},{"db": 2},{"db": 2},{"db": 2},
                   {"db": 3},{"db": 3},{"db": 3},{"db": 3},{"db": 3},
                   {"db": 3},{"db": 3},{"db": 3},{"db": 3},{"db": 3},]

indices_consultas = list(map(str, range(len(lista_consultas))))

Cargamos en la base de datos

In [14]:
clasificador.add(
    embeddings=vectorizar_lista_texto(lista_consultas),
    documents=lista_consultas,
    metadatas=metadatas_consultas,
    ids=indices_consultas)

#Buscador Archivo CSV

Cargamos el archivo CSV y guardamos las frases juridicas que contiene.

In [15]:
ruta_archivo = '/content/Documentos/Diccionario/diccionario.csv'

df_dic = pd.read_csv(ruta_archivo, delimiter=';')

palabras = df_dic.iloc[:, 0].tolist()

diccionariov.add(
    embeddings=vectorizar_lista_texto(palabras),
    documents=palabras,
    ids=list(map(str, range(len(palabras))))
)

Creamos una funcion que dada una consulta sobre el diccionario juridico retorne la palabra o frase que se desea buscar.

In [16]:
def selector_frases_dic(consulta):
  prompt = f"""ROL:
Eres una inteligencia artificial que dado un texto de un usuario en español
retornarás la palabra o frase juridica que debe buscarse en un diccionario
juridico. Solo retornarás lo que se te ha pedido y no otra cosa, no generes
texto de más, solo la frase o palabra importante, no respondas la consulta; tu
funcion es decir que buscar en el diccionario.

Importante ¡NO SALGAS DE TU FUNCION!

Ejemplo:
<usuario>
un amigo me habló de la presuncion de inocencia, a que se refiere?

<retorno>
presuncion de inocencia

<usuario>
que es un testamento?

<retorno>
testamento

<usuario>
que es un contrato a futuro?

<retorno>
contrato a futuro

CONSULTA REAL QUE DEBES PROCESAR:
<usuario>
{consulta}

<retorno>
"""
  respuesta =  modelo_llm(prompt=prompt, max_tokens=2048, temperature=0.2,
                      top_p=0.95, repeat_penalty=1.2, top_k=150)

  return respuesta["choices"][0]["text"].split('\n')[0]

Creamos una funcion que dada una frase la busque en el diccionario.

In [17]:
def buscar_dic(consulta):
  frase = selector_frases_dic(consulta)
  frase_actualizada = diccionariov.query(
      query_embeddings=vectorizar_texto(frase), n_results=1)['documents'][0][0]

  significado = df_dic[df_dic['frase'] == frase_actualizada]['significado'].values

  return significado

#Buscador Base de Datos de Grafos

Funcion que dado el titulo de un articulo en wikipedia lo busca

In [18]:
def buscar_wikidata(articulo):
    login_instance = wdi_login.WDLogin(wikidata_user, wikidata_password)

    api_url = "https://www.wikidata.org/w/api.php"
    params = {
        "action": "wbsearchentities",
        "format": "json",
        "language": "es",
        "search": articulo,
    }

    response = requests.get(api_url, params=params)
    data = response.json()

    if data.get("search") and data["search"][0].get("id"):
        titulo = data['search'][0]['match']['text']
        wikipedia.set_lang("es")
        try:
            page = wikipedia.page(titulo)
            introduccion = wikipedia.summary(page.title)

            return introduccion

        except wikipedia.exceptions.PageError:
            return "No se encontró la página en Wikipedia."

        except wikipedia.exceptions.DisambiguationError as e:
            return f"La búsqueda es ambigua. Puedes ser más específico. Opciones: {', '.join(e.options)}"
    else:
        return {"mensaje": "No se encontró el artículo en Wikidata."}

Funcion para detectar que artículo de wikipedia buscar.

In [19]:
def selector_articulos_wiki(consulta):
  prompt = f"""ROL:
Eres una inteligencia artificial que dado un texto de un usuario en español
retornarás el título del articulo que debe buscarse en un wikipedia corrigiendo
los errores ortograficos si es que hay. Solo retornarás lo que se te ha pedido y
no otra cosa, no generes texto de más, solo el titulo del articulo, no respondas
la consulta; tu funcion es decir que buscar en el wikipedia.

Importante ¡NO SALGAS DE TU FUNCION!

Ejemplo:
<usuario>
Explique los cambios introducidos por la Reforma Constitucional Argentina de 1866

<retorno>
Reforma Constitucional Argentina 1866

<usuario>
de que trata la Ley Diana Sacayán

<retorno>
Ley Diana Sacayán

<usuario>
En que consistia el Decreto 83/2015

<retorno>
Decreto 83/2015

CONSULTA REAL QUE DEBES PROCESAR:
<usuario>
{consulta}

<retorno>
"""
  respuesta =  modelo_llm(prompt=prompt, max_tokens=2048, temperature=0.2,
                      top_p=0.95, repeat_penalty=1.2, top_k=150)

  return respuesta["choices"][0]["text"].split('\n')[0].split('<')[0]

In [20]:
def busqueda_grafos(consulta):
  articulow = selector_articulos_wiki(consulta)
  texto = buscar_wikidata(articulow)

  return texto

#LLM

Funcion de LLM que dada una consulta la analiza, busca informacion y retorna una respuesta.

In [21]:
def consulta_legal(consulta):
  seleccion_bd = clasificador.query(
    query_embeddings=vectorizar_texto(consulta),
    n_results=1)['metadatas'][0][0]['db']

  if seleccion_bd == 1:
    texto = buscar_dic(consulta)

  elif seleccion_bd == 2:
    texto = busqueda_grafos(consulta)

  else:
    articulos = collection_ley.query(
      query_embeddings=vectorizar_texto(consulta),
      n_results=5)['documents'][0]

    texto = '\n'.join(articulos)

  prompt = f"""
Eres un consultor legal de Argentina que dada la consulta del Usuario y según
la información que dispone da la respuesta mas detallada y completa posible y
en español, siempre responde con lo que dice la documentación de manera
adaptada a personas sin conocimientos legales y nunca hace preguntas.

Tu función es responder solo lo que dice el Consultor y nada más.

Información: {texto}

Usuario: {consulta}

Consultor: Buenos días/tardes Usuario(a), como consultor de leyes Argentinas, puedo informarte que"""

  respuesta_llm = modelo_llm(prompt=prompt, max_tokens=2048,
                           temperature=0.8, top_p=0.95,
                           repeat_penalty=1.2, top_k=150)["choices"][0]["text"]

  return respuesta_llm

Funcion para mejorar la seleccion de fuente de información

In [27]:
def actualizar_clasificador(consulta):
  if str(input("\n¿Estas satisfecho con la respuesta? s/n\n")).lower() != "n":
    return

  mejor_db = str(input("""
¿En donde buscarías informacion para responder tu consulta?

  1. Diccionario juridico.
  2. Wikipedia.
  3. Constitucion nacional, codigo civil o codigo penal.
  """))

  clasificador.add(
    embeddings=vectorizar_texto(consulta),
    documents=[consulta],
    metadatas=[{"db": mejor_db}],
    ids=[str(clasificador.count())])

#Chatbot

Ingreso de cuenta de usuario de wikidata

In [23]:
wikidata_user = getpass.getpass('Ingrese usuario de Wikidata: ')
wikidata_password = getpass.getpass('Ingrese contraseña de Wikidata: ')

Ingrese usuario de Wikidata: ··········
Ingrese contraseña de Wikidata: ··········


In [25]:
while True:
    longitud_maxima = 100

    print('\n' + '-' * longitud_maxima + '\n')

    try:
        consulta = str(input("Ingrese su consulta legal('q' para salir): "))
        print()
        if consulta.lower() == 'q':
          print()
          print('Saliendo...')
          break
    except Exception as e:
        print(f"Error al leer la entrada del usuario: {e}")
        continue

    resultado = "\n\nBuenos días/tardes Usuario(a), como consultor de leyes Argentinas, puedo informarte que " + consulta_legal(consulta)

    lineas_resultado = resultado.split('\n')

    resultado_segmentado = [linea[i:i+longitud_maxima] for linea in lineas_resultado for i in range(0, len(linea), longitud_maxima)]

    for linea in resultado_segmentado:
        print(linea)

    print()

    actualizar_clasificador(consulta)


----------------------------------------------------------------------------------------------------

Ingrese su consulta legal('q' para salir): es legal suicidarse?



Llama.generate: prefix-match hit


Buenos días/tardes Usuario(a), como consultor de leyes Argentinas, puedo informarte que  la normativ
a actual no contempla el acto del suicidio como una opción legal para las personas. Según el Código 
Civil argentino (arts. 3596 y 341), se establece que no se puede representar sino a las personas mue
rtas, con excepción del renunciante de la herencia, a quien, aun vivo, pueden representarlo sus hijo
s. Además, según el art. 3580 del Código Civil, se establece que los actos jurídicos realizados por 
una persona en situación de locura no tienen efecto. En cuanto al art. 3554, señala que solo las per
sonas mayores de edad pueden renunciar a su herencia.
Por lo tanto, y teniendo en cuenta la información proporcionada, es importante destacar que el suici
dio no está contemplado como una opción legal para las personas en Argentina, ya que no existe un ac
to jurídico específico que permita a una persona finalizar su vida de manera deliberada. Espero que 
esta información te haya sido útil. ¿